<div style=" background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px 0; color:#fff; text-align: center"> Timestamp conversion
</h2>
</div>

In [1]:
import biblio_modules

from datetime import datetime
from dateutil import parser
import time
import pytz

import pyarrow as py
import pandas as pd
import numpy as np

In [2]:
transactions_processing = pd.read_csv("../Source/Transactions.csv", sep=";")
transactions_processing

,id_prod,date,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


<div style="background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Traitement Timestamp  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Début du data processing  </h3>
</div>

In [164]:
def split_date_time(df, column):
    df[['Date','Time']] = df[column].str.split(expand=True)
    return df

transactions_processing = split_date_time(transactions_processing, 'date').drop('date', axis=1)
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     687534 non-null  object
 1   session_id  687534 non-null  object
 2   client_id   687534 non-null  object
 3   Date        687534 non-null  object
 4   Time        687534 non-null  object
dtypes: object(5)
memory usage: 26.2+ MB


In [165]:
def convert_date_to_datetime(df):
    df['Date'] = pd.to_datetime(df['Date'])
    return df
transactions_processing = convert_date_to_datetime(transactions_processing)

condition = transactions_processing['Time'].str.startswith('24')
transactions_processing.loc[condition, 'Date'] = transactions_processing.loc[condition, 'Date'] + pd.DateOffset(days=1)
transactions_processing


,id_prod,session_id,client_id,Date,Time
0,0_1259,s_1,c_329,2021-03-01,00:01:07.843138
1,0_1390,s_2,c_664,2021-03-01,00:02:26.047414
2,0_1352,s_3,c_580,2021-03-01,00:02:38.311413
3,0_1458,s_4,c_7912,2021-03-01,00:04:54.559692
4,0_1358,s_5,c_2033,2021-03-01,00:05:18.801198
...,...,...,...,...,...
687529,1_508,s_348444,c_3573,2023-02-28,23:49:03.148402
687530,2_37,s_348445,c_50,2023-02-28,23:51:29.318531
687531,1_695,s_348446,c_488,2023-02-28,23:53:18.929676
687532,0_1547,s_348447,c_4848,2023-02-28,23:58:00.107815


In [166]:
def replace_24_hour_format(df, time_column):
    condition = df[time_column].str.startswith('24')
    df.loc[condition, time_column] = '00' + df.loc[condition, time_column].str[2:]
    return df

transactions_processing = replace_24_hour_format(transactions_processing, 'Time')
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     687534 non-null  object        
 1   session_id  687534 non-null  object        
 2   client_id   687534 non-null  object        
 3   Date        687534 non-null  datetime64[ns]
 4   Time        687534 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 26.2+ MB


In [167]:
#Je reconvertis mes date en string
def convert_column_to_string(df, column_name):
    df[column_name] = df[column_name].astype(str)
    return df

transactions_processing = convert_column_to_string(transactions_processing, 'Date')

In [168]:
def concat_date_time(df,timestamp_col, date_column, time_column):
    transactions_processing[timestamp_col] = transactions_processing[date_column].str.cat(transactions_processing[time_column], sep=' ')
    return df

transactions_processing = concat_date_time(transactions_processing, 'Timestamp', 'Date', 'Time').drop(['Date', 'Time'], axis=1)

In [169]:
transactions_processing['Timestamp'] = pd.to_datetime(transactions_processing['Timestamp'])
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     687534 non-null  object        
 1   session_id  687534 non-null  object        
 2   client_id   687534 non-null  object        
 3   Timestamp   687534 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 21.0+ MB


In [170]:
transactions_processing

,id_prod,session_id,client_id,Timestamp
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198
...,...,...,...,...
687529,1_508,s_348444,c_3573,2023-02-28 23:49:03.148402
687530,2_37,s_348445,c_50,2023-02-28 23:51:29.318531
687531,1_695,s_348446,c_488,2023-02-28 23:53:18.929676
687532,0_1547,s_348447,c_4848,2023-02-28 23:58:00.107815


In [171]:
def convert_to_local_time(df, timestamp_col):
    df = df.copy()    
    # Trouver les indices de la deuxième occurrence de chaque timestamp en double
    duplicated = df[timestamp_col].duplicated(keep='first')
    
    # Ajouter une seconde à la deuxième occurrence de chaque timestamp en double
    df.loc[duplicated, timestamp_col] = df.loc[duplicated, timestamp_col] + pd.Timedelta(seconds=1)
    
    return df

# Utilisation de la fonction
transactions_processing = convert_to_local_time(transactions_processing, 'Timestamp')
transactions_processing.loc[transactions_processing['Timestamp'].duplicated(keep=False)]

,id_prod,session_id,client_id,Timestamp


In [174]:
transactions_processing.head()

,id_prod,session_id,client_id,Timestamp
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198


In [176]:
transactions_processing.Timestamp.dtype

dtype('<M8[ns]')

In [177]:
#Je reorganise mes colonnes
def reorder_columns(df, new_order):
    df = df[new_order]
    return df

new_order = ['id_prod', 'Timestamp', 'session_id', 'client_id']
transactions_processing = reorder_columns(transactions_processing, new_order)

In [178]:
transactions_processing

,id_prod,Timestamp,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


In [181]:
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     687534 non-null  object        
 1   Timestamp   687534 non-null  datetime64[ns]
 2   session_id  687534 non-null  object        
 3   client_id   687534 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 21.0+ MB


In [182]:
def save_to_csv(df, file_path, sep=";", index=False):
    df.to_csv(file_path, sep=sep, index=index)
    
save_to_csv(transactions_processing, "../Source/Data_processed/timestamps_processed.csv")

<div style="background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Traitement Timestamp  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Fin du data processing  </h3>
</div>